# Generate Podcast Synopsis
- TO DO: give a few options

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load Data

In [2]:
fname = "../data/transcription.txt"

with open(fname, 'r') as f:
    content = f.readlines()

# convert list to str
content = ' '.join(content) 
print(content)

Title: Why SVB’s collapse is not a 2008 repeat
 The FT’s US financial commentator Robert Armstrong tells us why he’s not panicking
 
 Michela Tindera
 So, general question here, I mean, seeing bank runs . . . 
 
 Robert Armstrong
 Yes.
 
 Michela Tindera
 . . . just that term, I mean, like, yikes. Should we be freaking out?
 
 Robert Armstrong
 Mm-hmm. The problem, you know, you put me in a difficult situation, right? Because there’s a feedback loop here. Right? If everyone else is freaking out, you should freak out. But it would be much better if everyone didn’t freak out.
 
 Michela Tindera
 That’s the FT’s US financial commentator Rob Armstrong. And he’s been thinking about panic a lot recently. That’s because he, like a lot of financial journalists, has been watching a story over the last few days where panic has played a central role. And that story I’m talking about is the collapse of Silicon Valley Bank. And as this news broke last week, the headlines have been dire.
 
 News cli

## Set Up Azure OpenAI

In [3]:
import os
import openai
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

True

## Deploy a Model

In [4]:
# id of desired_model
desired_model = 'text-davinci-003' # suitable for text generation
desired_capability = 'completion'

# list models deployed with
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])

    # check if desired_model is deployed, and if it has 'completion' capability
    if model["id"] == desired_model and model['capabilities'][desired_capability]:
        deployment_id = deployment["id"]
        
# if no model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')

    # Deploy the model
    print(f'Creating a new deployment with model: {desired_model}')
    result = openai.Deployment.create(model=desired_model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {desired_model} that supports text {desired_capability} with id: {deployment_id}.')
else:
    print(f'Found a succeeded deployment of "{desired_model}" that supports text {desired_capability} with id: {deployment_id}.')

Found a succeeded deployment of "text-davinci-003" that supports text completion with id: text-davinci-003.


## Text chunks generator

In [5]:
# A generator that split a text into smaller chunks of size n, preferably ending at the end of a sentence
def chunk_generator(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


## Request API

In [24]:
def request_api(document, prompt_postfix, max_tokens):
    prompt = prompt_postfix.replace('<document>',document)
    #print(f'>>> prompt : {prompt}')

    response = openai.Completion.create(  
    deployment_id=deployment_id, 
    prompt=prompt,
    temperature=0,
    max_tokens=max_tokens,
    top_p=1,
    frequency_penalty=1,
    presence_penalty=1,
    stop='###')

    return response['choices'][0]['text']

## Generate Synopsis

In [7]:
# Prompt postfix
prompt_postfix = """ <document>
  \n###
  \nSummarise the transcript of a podcast above into a synopsis. 
  \nSynopsis : 
"""
print(prompt_postfix)

 <document>
  
###
  
Summarise the transcript of a podcast above into a synopsis. 
  
Synopsis : 



In [8]:
import tiktoken

synopsis_chunck = []
n = 2000 # max tokens for chuncking
max_tokens = 1000 # max tokens for response

tokenizer = tiktoken.get_encoding('p50k_base')

# Generate chunkcs    
chunks = chunk_generator(content, n, tokenizer)

# Decode chunk of text
text_chunks = [tokenizer.decode(chunk) for chunk in chunks]

# Request api
for chunk in text_chunks:
    synopsis_chunck.append(request_api(chunk, prompt_postfix, max_tokens))
    #print(chunk)
    #print('>>> synopsis: \n' + synopsis_chunck[-1])

# Synopsis
synopsis = ' '.join(synopsis_chunck)
print(synopsis)

The Financial Times' US financial commentator Robert Armstrong discusses the collapse of Silicon Valley Bank and why it is not a repeat of the 2008 financial crisis. He explains that two factors led to SVB's collapse: bad decisions at the bank and a rapid increase in interest rates. Rob also outlines how banks operate, what went wrong with SVB specifically, and whether there are larger systemic reasons for this. He further explains why we have a two-tiered system when it comes to banking regulations, as well as how Dodd-Frank Act rollback may or may not have played a role in this situation. Finally, he reassures listeners that so long as people don't panic, everything should be fine and that deposits under $250k are covered by the US government. Robert Armstrong, a financial expert, discusses the recent collapse of SVB and its implications for banking regulation. He explains that banks are now better capitalised than before 2008 due to regulations put in place after the crisis. He also

## Translate Synopsis

In [9]:
# Prompt postfix
prompt_postfix = """ <document>
  \n###
  \nTranslate synopsis into Mandarin.  
  \nTranslation : 
"""
print(prompt_postfix)

 <document>
  
###
  
Translate synopsis into Mandarin.  
  
Translation : 



In [10]:
max_tokens = 1000
translation = request_api(synopsis, prompt_postfix, max_tokens)
print(translation)

美国金融评论家罗伯特·阿姆斯特朗（Robert Armstrong）在《金融时报》上就硅谷银行的崩溃进行了讨论，并且说明这不是2008年金融危机的复制。他解释说，导致SVB倒闭的原因有两方面：一是银行内部出现了不当决定; 二是利率急剧上升。此外，Rob 还概述了银行如何运作、SVB具体出现什么问题以及是否存在更大的体系性原因。此外，他还详细说明了为什么我们在监管方面存在双标准制度, 以及道德弗兰克法案(Dodd-Frank Act) 的退减对此情况会不会造成影响。最后,  Robert Armstrong 向听众保证, 只要人民不惊慌失措, 情况应当一切安好, 250k 美元之下的储户由美国政府承保.


美国金融评论家罗伯特·阿姆斯特朗（Robert Armstrong）在《金融时报》上就硅谷银行的崩溃进行了讨论，并且说明这不是2008年金融危机的复制。他解释说，导致SVB倒闭的原因有两方面：一是银行内部出现了不当决定; 二是利率急剧上升。此外，Rob 还概述了银行如何运作、SVB具体出现什么问题以及是否存在更大的体系性原因。此外，他还详细说明了为什么我们在监管方面存在双标准制度, 以及道德弗兰克法案(Dodd-Frank Act) 的退减对此情况会不会造成影响。最后,  Robert Armstrong 向听众保证, 只要人民不惊慌失措, 情况应当一切安好, 250k 美元之下的储户由美国政府承保.
Robert Armstrong 金融专家提出了 SVB 最新崩盘及其对相关监管带来的影响。 他表明由于 2008 年危机之后所施加的法律法规使得如今的银行已然比 2008 年时更加强壮。 此外， 也告诫投者将存款存入需要留意看看能耐性情况。 最后 ， Robert Armstrong 预测随之考勒将使得 bank equity capital 更加昂���耗时曲緩效应将随之考勒使效能受到影响.

## Generate Tag Lines

In [11]:
# Prompt postfix
prompt_postfix = """ <document>
  \n###
  \nGenerate 2 to 3 tag lines based on the podcast synopsis above.
"""
print(prompt_postfix)

 <document>
  
###
  
Generate 2 to 3 tag lines based on the podcast synopsis above.



In [12]:
max_tokens = 500
tag_lines = request_api(synopsis, prompt_postfix, max_tokens)
print(tag_lines)


1. Get the facts on Silicon Valley Bank's collapse and its implications for banking regulation.
2. Don't panic: Robert Armstrong explains why deposits under $250k are safe.
3. Learn how Dodd-Frank Act rollback may have played a role in SVB's collapse with Robert Armstrong.


## Generate Search Engine Optimised (SEO) Keywords

In [22]:
# Prompt postfix
prompt_postfix = """ <document>
  \n###
  \nGenerate 5 search engine optimised keywords based on text above.  
"""
print(prompt_postfix)

 <document>
  
###
  
Generate 5 search engine optimised keywords based on text above.  



In [25]:
keywords_chunck = []
n = 2000 # max tokens for chuncking
max_tokens = 100

tokenizer = tiktoken.get_encoding('p50k_base')

# Generate chunkcs    
chunks = chunk_generator(content, n, tokenizer)

# Decode chunk of text
text_chunks = [tokenizer.decode(chunk) for chunk in chunks]

# Request api
for chunk in text_chunks:
    keywords_chunck.append(request_api(chunk, prompt_postfix, max_tokens))

# Keywords
keywords = ' '.join(keywords_chunck)
print(keywords)


1. Silicon Valley Bank collapse
2. 2008 Financial Crisis
3. US Banking Regulation
4. FDIC Resolution 
5. Treasury Liquidity Support 
1. Banking Regulation
2. Financial Crisis 2008
3. Bank Equity Capital 
4. SVB Collapse 
5. Maturity Transformation


In [28]:
# Prompt postfix
prompt_postfix = """ <document>
  \n###
  \nReduce keywords above to 5 search engine optimised keywords. 
"""
print(prompt_postfix)

 <document>
  
###
  
Reduce keywords above to 5 search engine optimised keywords. 



In [30]:
max_tokens = 50
seo_keywords = request_api(keywords, prompt_postfix, max_tokens)
print(seo_keywords)


1. Banking Regulation
2. Financial Crisis 2008
3. Bank Equity Capital 
4. SVB Collapse 
5. Treasury Liquidity Support
